In [ ]:
!pip install -qr /content/drive/MyDrive/MajorProj/cnn_yolo/yolov5/requirements.txt

In [ ]:
early_blight_dataset_path="/content/drive/MyDrive/MajorProj/Dataset/Tomato_Early_blight"
yellow_curl_dataset_path="/content/drive/MyDrive/MajorProj/Dataset/Tomato__Tomato_YellowLeaf__Curl_Virus"
healthy_path="/content/drive/MyDrive/MajorProj/Dataset/ORGINAL_PLANT_VILLAGE/Tomato_healthy"
bacterial_path="/content/drive/MyDrive/MajorProj/Dataset/ORGINAL_PLANT_VILLAGE/Tomato_Bacterial_spot"

In [ ]:
def labelImageAndReturnLabelFile(imagePath):
  #detect label
  res = !python /content/drive/MyDrive/MajorProj/cnn_yolo/yolov5/detect.py --save-txt --weights /content/drive/MyDrive/MajorProj/cnn_yolo/model.pt --source '{imagePath}' --img 416 --conf 0.4
  #grab ouptut label path
  labelOp='/content/'+res[-1].split()[-1]+'/'
  textFileWithDimension=""
  for root, dirs, files in os.walk(labelOp):
      for file in files:
          if file.endswith('.txt'):
              print(file)
              textFileWithDimension=f'{labelOp}{file}'
              break;
  return textFileWithDimension

def convertYoloCoordinateToOpenCv(imagePath,yoloTextFile):
  img = cv2.imread(imagePath)
  dh, dw, _ = img.shape

  fl = open(yoloTextFile,"r")
  data = fl.readlines()
  fl.close()

  for dt in data:

      # Split string to float
      _, x, y, w, h = map(float, dt.split(' '))
      l = int((x - w / 2) * dw)
      r = int((x + w / 2) * dw)
      t = int((y - h / 2) * dh)
      b = int((y + h / 2) * dh)
      
      if l < 0:
          l = 0
      if r > dw - 1:
          r = dw - 1
      if t < 0:
          t = 0
      if b > dh - 1:
          b = dh - 1

      cv2.rectangle(img, (l, t), (r, b), (0, 0, 255), 1)
      return (l,t,r,b)


def extractImagePart(orginalImage,l,t,r,b):
  orginal=cv2.imread(orginalImage)
  cropped_img = orginal[t:b,l:r]
  return cropped_img

In [ ]:
import os
import os.path
import cv2
from google.colab.patches import cv2_imshow
# import cv2_imshow
for root, dirs, files in os.walk(bacterial_path):
   count=0
   for name in files:
      imgPath=os.path.join(root, name)
      destPath="/content/drive/MyDrive/MajorProj/Dataset/CROPPED/Bacterial/"+"cropped_"+name
      try:
        with open(destPath) as f:
          print("Already 😇"+ destPath)
      except: 
        labelFile=labelImageAndReturnLabelFile(imgPath)
        try:
          (l,t,r,b)=convertYoloCoordinateToOpenCv(imgPath,labelFile)
        except:
          continue
              
        cropped=extractImagePart(imgPath,l,t,r,b)
      
        cv2.imwrite(destPath,cropped)
        print("✔️✔️ "+"Index: "+str(count)+" "+name)
        count=count+1
